# Calculation of velocity model from the B-spline coefficients

This notebook reads the output files resulting from the inversion using the SIM30 code and it provides the P and S velocities at any point within the target volume. The notebook also provides the possibility to generate arbitrary cross-sections given an initial point (lat,lon), the bearing (degrees clockwise from N) and the length in km. The output of the cross sections is provided as a csv file. Similarly, it is possible to use the hit-count file (fort.83) to provide the "resolution" level for the model

In [1]:
import os
import numpy as np
import pandas as pd
import math

In [2]:
from geopy.distance import geodesic
from pyproj import Geod

In [3]:
def bsplin(h, x, n, i):
    # calculates the factor fx for a given position
# 
    fx = 0.0

    if h < 0.0001:
        fx = 1.0
        return fx

    h1 = 1.0 / h
    h2 = h1 * h1
    h3 = h2 * h1

    if i == 1 and x < h:
        fx = 0.25 * ((h3 * x * x - 6.0 * h1) * x + 6.0)
        return fx
    if i == 2 and x < h:
        fx = 0.5 * (-h3 * x * x + 3.0 * h1) * x
        return fx
    if i == (n - 1) and x > (n - 2) * h:
        aux = x - (n - 2) * h
        fx = 0.5 * (h3 * aux - 3.0 * h2) * aux * aux + 1.0
        return fx
    if i == n and x > (n - 2) * h:
        aux = x - (n - 2) * h
        fx = 0.25 * (((-h3 * aux + 3.0 * h2) * aux + 3.0 * h1) * aux + 1.0)
        return fx
    if x >= (i - 3) * h and x <= (i - 2) * h:
        aux = x - (i - 3) * h
        fx = 0.25 * h3 * aux * aux * aux
        return fx
    if x >= (i - 2) * h and x <= (i - 1) * h:
        aux = x - (i - 2) * h
        fx = 0.25 * (((-3.0 * h3 * aux + 3.0 * h2) * aux + 3.0 * h1) * aux + 1.0)
        return fx
    if x >= (i - 1) * h and x <= i * h:
        aux = x - (i - 1) * h
        fx = 0.75 * (h3 * aux - 2.0 * h2) * aux * aux + 1.0
        return fx
    if x >= i * h and x <= (i + 1) * h:
        aux = x - i * h
        fx = 0.25 * (((-h3 * aux + 3.0 * h2) * aux - 3.0 * h1) * aux + 1.0)
        return fx

    return fx

In [4]:
def vel3m(isp, x, y, z, nx, ny, nz, xn, yn, zn, acfp, acfs):
    # This function determines the velocity (P or S) at a point (x,y,z)
    # using the model defined by cubic B-splines with coefficients defined
    # at each node through the arrays acfp and acfs. The grid has nx,ny,nz nodes
    # defined at the points xn[0:nx], yn[0:ny], zn[0:nz]
    # 
    #
    x1 = x
    y1 = y
    z1 = z

    x1 = x1 - xn[0]
    y1 = y1 - yn[0]
    z1 = z1 - zn[0]

    # determine max values of the grid
    xout = float(nx - 1) * hx
    yout = float(ny - 1) * hy
    zout = float(nz - 1) * hz

    if x1 < 0:
        x1 = 0
    if x1 >= xout:
        x1 = xout
    if y1 < 0:
        y1 = 0
    if y1 >= yout:
        y1 = yout
    if z1 < 0:
        z1 = 0
    if z1 >= zout:
        z1 = zout

    # set the B-spline values to 0
    fx = np.zeros(nx)
    fy = np.zeros(ny)
    fz = np.zeros(nz)

    for i in range(nx):
        fx[i] = bsplin(hx, x1, nx, i + 1)

    for i in range(ny):
        fy[i] = bsplin(hy, y1, ny, i + 1)

    for i in range(nz):
        fz[i] = bsplin(hz, z1, nz, i + 1)

    # Calculation of velocity using splines
    v = 0

    if isp == 1:
        for i in range(nx):
            if fx[i] == 0:
                continue
            for j in range(ny):
                if fy[j] == 0:
                    continue
                for k in range(nz):
                    v += acfs[i, j, k] * fx[i] * fy[j] * fz[k]
    else:
        for i in range(nx):
            if fx[i] == 0:
                continue
            for j in range(ny):
                if fy[j] == 0:
                    continue
                for k in range(nz):
                    v += acfp[i, j, k] * fx[i] * fy[j] * fz[k]

    return v

In [5]:
# Step 2: Function to compute Cartesian coordinates
def latlon_to_xy(latitude, longitude, origin_lat, origin_lon, rotation_angle_deg):
    # Calculate distances in kilometers from origin (northing/easting)
    north_dist = geodesic((latitude, longitude), (origin_lat, longitude)).kilometers
    east_dist = geodesic((latitude, longitude), (latitude, origin_lon)).kilometers

    # Adjust signs based on the relative position
    if latitude < origin_lat:
        north_dist *= -1
    if longitude < origin_lon:
        east_dist *= -1

    # Convert rotation angle to radians
    rotation_angle_rad = np.radians(rotation_angle_deg)

    # Apply rotation transformation
    x = east_dist * np.cos(rotation_angle_rad) - north_dist * np.sin(rotation_angle_rad)
    y = east_dist * np.sin(rotation_angle_rad) + north_dist * np.cos(rotation_angle_rad)

    return x, y

In [6]:
def latlon_to_xy_inverted(latitude, longitude, origin_lat, origin_lon, rotation_angle_deg):
    # Calculate distances in kilometers from origin (northing/easting)
    north_dist = geodesic((latitude, longitude), (origin_lat, longitude)).kilometers
    east_dist = geodesic((latitude, longitude), (latitude, origin_lon)).kilometers

    # Adjust signs based on the relative position
    if latitude < origin_lat:
        north_dist *= -1
    if longitude < origin_lon:
        east_dist *= -1

    # Convert rotation angle to radians
    rotation_angle_rad = np.radians(rotation_angle_deg)

    # Apply rotation transformation with inverted x-axis
    x = -1 * (east_dist * np.cos(rotation_angle_rad) - north_dist * np.sin(rotation_angle_rad))  # Invert x
    y = east_dist * np.sin(rotation_angle_rad) + north_dist * np.cos(rotation_angle_rad)

    return x, y

In [7]:
def track_points(initial_latitude,initial_longitude,track_length_km,bearing_degrees,point_spacing_km):
    # Step 2: Initialize the Geod instance (WGS84 ellipsoid)
    geod = Geod(ellps="WGS84")

    # Step 3: Calculate the waypoints
    # Number of points to generate
    num_points = int(track_length_km / point_spacing_km) + 1

    # Generate distances along the track (equally spaced)
    distances = np.linspace(0, track_length_km * 1000, num_points)  # Convert to meters
    
    distances_km = distances/1000.

    # Calculate the waypoints
    waypoints = [
        geod.fwd(initial_longitude, initial_latitude, bearing_degrees, distance)
        for distance in distances
    ]

    # Extract latitude and longitude of waypoints
    lats, lons = zip(*[(lat, lon) for lon, lat, _ in waypoints])
    return lons,lats, distances_km

In [8]:
def read_model(input_file):
    # read input file define by cubic B-splines
    with open(input_file, 'r') as f:
        nx, ny, nz = map(int, f.readline().split())
        x0, hx, y0, hy, z0, hz = map(float, f.readline().split())
        #
        xn = np.zeros(nx)
        yn = np.zeros(ny)
        zn = np.zeros(nz)
    
        for i in range(nx):
            xn[i] = x0 + hx*i
        #
        for i in range(ny):
            yn[i] = y0 + hy*i

        for i in range(nz):
            zn[i] = z0 + hz*i
    
        data = np.zeros((nx, ny, nz))

        for i in range(nx):
            for j in range(ny):
                data[i, j, :] = list(map(float, f.readline().split()))
    
        acfp = data.copy()  

    
        for i in range(nx):
            for j in range(ny):
                data[i, j, :] = list(map(float, f.readline().split()))
    
        acfs = data.copy()  
        return acfp,acfs,nx,ny,nz,xn,yn,zn,hx,hy,hz

In [13]:
# Define a step function
def step_function(x,thresh):
    if x > thresh:
        return 1
    else:
        return 0

# Start

## Part 1: calculation of P and S velocities at an arbitrary point in the target volume

In [14]:
# input the file with the B-spline coefficients
#
#input_file = "BSI-R_fort.43" # INGV Bulletin
#input_file = "PN-OR_fort.43" # PhaseNet original
input_file = "PN-IN_fort.43" # PhaseNet INSTANCE

## input file structure
The input file reads for each row all the depths. The outer loop is according to the x-coordinate. The first
line corresponds to the 0,0,[:] locations. The second row to the 0,1,[:], ... the ny row corresponds the new 
position of the x-coordinate. Thus if nx=17, ny=16 and nz=7, the position 1,0,[:] corresponds to row 16 (starting 
from 0!]

In [15]:
acfp,acfs,nx,ny,nz,xn,yn,zn,hx,hy,hz = read_model(input_file)

## Test calculation of velocity

In [16]:
isp=0 # 0 for P and 1 for S
x,y,z =(24.0,30.0,11.0)
#x,y,z = (12.48,30.0,11.0)

vel3m(isp, x, y, z, nx, ny, nz, xn, yn, zn, acfp, acfs)

6.499305

### The user can add below the calculation at any point and create horizontal and vertical slices through the model. An example of vertical cross-sections is given in Part 2 below

## Part 2: calculation of P and S velocities and of the resolution given the hit-counts for an arbitrary cross-section in the volume 

### Define the cross-section points in the model

In [17]:
#initial_latitude = 42.85 # section A
#initial_longitude = 12.87
#
initial_latitude = 42.78 # section B
initial_longitude = 12.925
#
#initial_latitude = 42.705 # section C
#initial_longitude = 12.97
#
# chiarabba et al (2020)
#initial_latitude = 42.78 # 
#initial_longitude = 12.87
#
track_length_km = 30  # Total length of the track in kilometers
bearing_degrees = 70  # Angle (clockwise from North) in degrees
#
# chiarabba et al.
#bearing_degrees = 77  # Angle (clockwise from North) in degrees
#
#point_spacing_km = 5.0  # Distance between points in kilometers
point_spacing_km = 0.25  # Distance between points in kilometers

In [18]:
lons,lats, distances_km = track_points(initial_latitude,initial_longitude,track_length_km,bearing_degrees,point_spacing_km)
#
print (f'Number of points along track: {len(distances_km)}')
print (f'first point: {initial_latitude}, {initial_longitude}    end point: {lats[-1]}, {lons[-1]} ')
#for i, (lat, lon) in enumerate(zip(lats, lons)):
#    print(f"Point {i}: Distance_km = {distances_km[i]:.2f}, Latitude = {lat:.6f}, Longitude = {lon:.6f}")


Number of points along track: 121
first point: 42.78, 12.925    end point: 42.87184430890089, 13.270009989795575 


## Mapping of the points along the track into x,y coordinates

In [19]:
# Step 1: Define the origin and rotation of the coordinate system used for the inversion
# the tomo inversion results used the following orgin of the coordinates (do not change!)
origin_lat = 42.833333
origin_lon = 13.125
rotation_angle_deg = 0  # Rotation angle in degrees (counterclockwise)

### Test the coordinates origin

In [20]:
latd,latm = (42,50.0)
lond,lonm = (13,7.50)
test_lat = latd + latm/60.
test_lon = lond + lonm/60.
print (test_lat,test_lon)

42.833333333333336 13.125


## generate cross-section

In [21]:
# set the dimensions of the cells along depth equal to those along the track
section_vert = zn.max() - zn.min()
n_vert = int(section_vert/point_spacing_km + 1)
section_hori = distances_km.max() - distances_km.min()
print (f'horizontal length: {section_hori}   vertical length: {section_vert}')
print (f'no. of points:  {len(distances_km)}  no. of points: {n_vert}')

horizontal length: 30.0   vertical length: 12.0
no. of points:  121  no. of points: 49


## generate the dataframe containing the velocities (P and S) of the selected cross-section

In [22]:
%%time
depths = np.linspace(zn[0], zn[-1], n_vert)

data = []
#
for isp in [0,1]:
    for k in range(len(depths)):
        z = depths[k]
        for i in range(len(lats)):
            x, y = latlon_to_xy_inverted(lats[i], lons[i], origin_lat, origin_lon, rotation_angle_deg)
            v = vel3m(isp, x, y, z, nx, ny, nz, xn, yn, zn, acfp, acfs)
            #
            row = [x,y,z,v, int(isp), distances_km[i], lats[i], lons[i]]
            data.append(row)
#
data_array = np.array(data)
# 
section_df = pd.DataFrame(data_array, columns=["x", "y", "z", "v", "v_type", "distance_km", "latitude", "longitude"]) 


CPU times: user 2.5 s, sys: 13.5 ms, total: 2.51 s
Wall time: 2.51 s


In [23]:
section_df["v_type"] = section_df["v_type"].astype(int)
num_digits = 5
#
fname = f"section_{initial_latitude}_{initial_longitude}_{bearing_degrees}.csv"
out_file = fname
section_df.to_csv(out_file, float_format=f"%.{num_digits}f", index=False)
#section_df.head(30)

In [24]:
section_df[section_df['z'] == 3]

,x,y,z,v,v_type,distance_km,latitude,longitude
1936,16.366252,-5.924708,3.0,6.000873,0,0.00,42.780000,12.925000
1937,16.131126,-5.839207,3.0,6.003473,0,0.25,42.780770,12.927871
1938,15.896001,-5.753714,3.0,6.006201,0,0.50,42.781539,12.930742
1939,15.660875,-5.668229,3.0,6.009088,0,0.75,42.782309,12.933613
1940,15.425749,-5.582752,3.0,6.012203,0,1.00,42.783078,12.936484
...,...,...,...,...,...,...,...,...
7981,-10.908296,3.939981,3.0,3.230568,1,29.00,42.868800,13.258493
7982,-11.143422,4.024552,3.0,3.198044,1,29.25,42.869561,13.261372
7983,-11.378547,4.109115,3.0,3.166211,1,29.50,42.870322,13.264251
7984,-11.613673,4.193669,3.0,3.136349,1,29.75,42.871083,13.267131


In [25]:
section_df.describe()

,x,y,z,v,v_type,distance_km,latitude,longitude
count,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000
mean,2.258725,-0.813703,5.000000,4.653498,0.500000,15.000000,42.826008,13.097421
std,8.212923,2.969905,3.535683,1.462254,0.500021,8.732493,0.026734,0.100427
min,-11.848798,-5.924708,-1.000000,2.268554,0.000000,0.000000,42.780000,12.925000
25%,-4.795035,-3.363156,2.000000,3.387724,0.000000,7.500000,42.803059,13.011157
50%,2.258724,-0.808811,5.000000,4.069639,0.500000,15.000000,42.826052,13.097377
75%,9.312485,1.738317,8.000000,6.282443,1.000000,22.500000,42.848981,13.183662
max,16.366252,4.278216,11.000000,6.742814,1.000000,30.000000,42.871844,13.270010


## determine the resolution through the hit counts for same section

In [26]:
#input_file = os.path.join(ROOTDIR,DATASOURCE,INVERSION,"fort.83")

#input_file = "BSI-R_fort.83" # INGV Bulletin
#input_file = "PN-OR_fort.83" # PhaseNet original
input_file = "PN-IN_fort.83" # PhaseNet INSTANCE

acfp,acfs,nx,ny,nz,xn,yn,zn,hx,hy,hz = read_model(input_file)

In [27]:
%%time
depths = np.linspace(zn[0], zn[-1], n_vert)

data = []
#
for isp in [0,1]:
    for k in range(len(depths)):
        z = depths[k]
        for i in range(len(lats)):
            x, y = latlon_to_xy_inverted(lats[i], lons[i], origin_lat, origin_lon, rotation_angle_deg)
            v = vel3m(isp, x, y, z, nx, ny, nz, xn, yn, zn, acfp, acfs)
            #
            row = [x,y,z,v, int(isp), distances_km[i], lats[i], lons[i]]
            data.append(row)
#
data_array = np.array(data)
# 
section_df = pd.DataFrame(data_array, columns=["x", "y", "z", "hit", "v_type", "distance_km", "latitude", "longitude"]) 
thresh = section_df['hit'].median()
section_df['hit_step'] = section_df['hit'].apply(lambda x: step_function(x, thresh))

CPU times: user 2.5 s, sys: 7.74 ms, total: 2.51 s
Wall time: 2.51 s


In [28]:
section_df["v_type"] = section_df["v_type"].astype(int)
num_digits = 5
#
fname = f"section_hit_{initial_latitude}_{initial_longitude}_{bearing_degrees}.csv"
#
out_file = fname
section_df.to_csv(out_file, float_format=f"%.{num_digits}f", index=False)
#section_df.head(30)

In [29]:
section_df.describe()

,x,y,z,hit,v_type,distance_km,latitude,longitude,hit_step
count,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000,11858.000000
mean,2.258725,-0.813703,5.000000,1.016992,0.500000,15.000000,42.826008,13.097421,0.500000
std,8.212923,2.969905,3.535683,0.732671,0.500021,8.732493,0.026734,0.100427,0.500021
min,-11.848798,-5.924708,-1.000000,0.009743,0.000000,0.000000,42.780000,12.925000,0.000000
25%,-4.795035,-3.363156,2.000000,0.406280,0.000000,7.500000,42.803059,13.011157,0.000000
50%,2.258724,-0.808811,5.000000,0.830611,0.500000,15.000000,42.826052,13.097377,0.500000
75%,9.312485,1.738317,8.000000,1.562697,1.000000,22.500000,42.848981,13.183662,1.000000
max,16.366252,4.278216,11.000000,2.826041,1.000000,30.000000,42.871844,13.270010,1.000000
